In [1]:
# declaraciones de librerías
using DataStructures

In [2]:
type gPaths
    nVertices::Int64
    arcos::Array{Int64,2} #arco[i,j] indica la distancia entre i y j 
end

type arcoPaths
    origen::Int64
    destino::Int64
    longitud::Int64
end

type gPathsSparse
    nVertices::Int64
    nArcos::Int64
    pOrigen::Array{Int64,1}
    arcos::Array{arcoPaths,1} #arco[i,j] indica la distancia entre i y j 
end

function generarGrafoDense(n::Int64,maxValue::Int64)
    a=Array{Int64}(n,n)
    rand!(a,1:maxValue)
    for i in 1:n
        a[i,i]=0
    end
    return gPaths(n,a) 
end

function generarGrafoSparse(n::Int64,sp::Float64,maxValue::Int64)
    a=arcoPaths[]
    pOrigen=Int64[]
    nArcos=1
    for i in 1:n
        push!(pOrigen,nArcos)
        for j in 1:n
            if i!=j
                if rand()<sp
                    push!(a,arcoPaths(i,j,rand(1:maxValue)))
                    nArcos += 1
                end
            end
        end
    end
    push!(pOrigen,nArcos)
    return gPathsSparse(n,nArcos-1,pOrigen,a) 
end

function fromDenseToSparse(g::gPaths)
    a=arcoPaths[]
    pOrigen=Int64[]
    nArcos=1
    for i in 1:g.nVertices
        push!(pOrigen,nArcos)
        for j in 1:g.nVertices
            if g.arcos[i,j]!=0
                push!(a,arcoPaths(i,j,g.arcos[i,j]))
                nArcos += 1
            end
        end
    end    
    push!(pOrigen,nArcos)        
    return gPathsSparse(g.nVertices,nArcos-1,pOrigen,a)
end

function fromSparseToDense(g::gPathsSparse)
    ∞=1000000000
    a=Array{Int64}(g.nVertices,g.nVertices)
    fill!(a,∞)
    for i in 1:g.nArcos
        a[g.arcos[i].origen,g.arcos[i].destino]=g.arcos[i].longitud
    end
    return gPaths(g.nVertices,a)
end

fromSparseToDense (generic function with 1 method)

# Djikstra

Implementación

<b>P1. Inicialización:</b> l(s):=0, l(v):=$\infty$ para todo $v\in V \setminus \{s\}$, R:=$\emptyset$

<b>P2. Buscar Probe:</b> Buscar un vértice $v\in V \setminus R$ tal que $l(v):=\min_{w\in V\setminus R} l(w)$

<b>P3. Update R:</b> Sea $R:=R\cup \{v\}$ 

<b>P4. actualizar:</b> 

Para todo $w\in V \setminus R$ tal que $(v,w)\in A$ hacer:<br>
$~~~$Si $l(w)>l(v)+c(v,w)$ entonces <br>
$~~~~~~$sea $l(w):=l(v)+c(v,w)$ y $\pi(w):=v$ <br>
$~~~$Fin Si <br>
Fin Para <br>

<b>P5. Condición de final:</b> Si $R\neq V$ ir a paso 2.

La implementación actual sólo sirve si no hay circuitos, ya que $R$ no se mantiene (una opción es contar el número de vértices considerados en el <b>paso 4</b>.

La implementación propuesta es suficientemente eficiente para la mayoría de problemas y la recomendada si los arcos no pueden tener valores negativos (excepto en la forma de guardar los arcos y de mirar el $\delta^+$ de un vértice que puede mejorarse). 

In [3]:
function dijkstraDense(g::gPaths,origin::Int64)
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origin]=0
    Q=PriorityQueue()
    enqueue!(Q,origin,0)
    while isempty(Q)==false
        v=dequeue!(Q)
        for w in 1:g.nVertices
            if l[w] > (l[v]+g.arcos[v,w])
                l[w]=l[v]+g.arcos[v,w]
                π[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
        end
    end
    return l,π
end

function dijkstraSparse(g::gPathsSparse,origin::Int64)
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origin]=0
    Q=PriorityQueue()
    enqueue!(Q,origin,0)
    while isempty(Q)==false
        v=dequeue!(Q)
        for c in g.pOrigen[v]:g.pOrigen[v+1]-1
            w=g.arcos[c].destino
            longitud=g.arcos[c].longitud
            if l[w] > (l[v]+longitud)
                l[w]=l[v]+longitud
                π[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
        end
    end
    return l,π
end

dijkstraSparse (generic function with 1 method)

In [57]:
G=generarGrafoDense(1000,1000);
@time(dijkstraDense(G,1));
#l=dijkstraDense(G,1)
#println("in: ",l)
gS=fromDenseToSparse(G);
@time(dijkstraSparse(gS,1));
#lS=dijkstraSparse(gS,1)
#println("in: ",lS)


  0.158427 seconds (1.54 M allocations: 23.770 MB, 2.42% gc time)
  0.569042 seconds (4.54 M allocations: 84.798 MB, 36.20% gc time)


In [5]:
gS=generarGrafoSparse(1000,0.1,1000);
@time(dijkstraSparse(gS,1));
G=fromSparseToDense(gS);
@time(dijkstraDense(G,1));

  0.039571 seconds (510.38 k allocations: 9.521 MB, 11.07% gc time)
  0.607310 seconds (2.46 M allocations: 37.791 MB, 4.15% gc time)


# Bellman-Ford

Alternativamente y si se cree que puede haber circuitos (o es lo que se busca) es preferible usar Bellman-Ford. La recurrencia (recordemos que los problemas de caminos son programas dinámicos) se reordena para analizar arcos. 

Analizando los arcos en un máximo de ($n$-1) comprobaciones, aseguramos que se exploran caminos de la longitud máxima posible y que si existe un circuito éste puede trazarse (no hay circuitos hamiltonianos con más de ese número de arcos).
Implementación

<b>P1. Inicialización:</b> l(s):=0, l(v):=$\infty$ para todo $v\in V \setminus \{s\}$

<b>P2. Iteración:</b> 

Para i=1,...,|V|-1<br>
$~~~$Para todo arco $(v,w)\in A$ <br>
$~~~~~~$Si $l(w)>l(v)+c(v,w)$ entonces <br>
$~~~~~~~~~$sea $l(w):=l(v)+c(v,w)$ y $\pi(w):=v$
$~~~~~~$Fin Si<br>
$~~~$Fin Para<br>
Fin Para

<b>P3. Circuitos:</b> Si existe un arco (v,w) tal que $l(w)>l(v)+c(v,w)$<br>
$~~~$Reportar que existe un circuito que empieza en $w$

In [28]:
function BellmanFordDense(g::gPaths,origin::Int64)
    #paso 1
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origin]=0
    #paso 2
    for i = 1:g.nVertices-1
        for v in 1:g.nVertices
            for w in 1:g.nVertices
                if l[w]> (l[v]+g.arcos[v,w])
                    l[w]=l[v]+g.arcos[v,w]
                    π[w]=v
                end
            end
        end
    end
    #paso 3
    conCircuito=0
    for v in 1:g.nVertices
        for w in 1:g.nVertices
            if l[w]> (l[v]+g.arcos[v,w])
                conCircuito=w
                break
            end
        end
    end
    return conCircuito,l,π
end

function BellmanFordSparse(g::gPathsSparse,origin::Int64)
    #paso 1
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origin]=0
    #paso 2
    for i = 1:g.nVertices-1
        for c in 1:g.nArcos
            v=g.arcos[c].origen
            w=g.arcos[c].destino
            longitud=g.arcos[c].longitud
            if l[w]> (l[v]+longitud)
                l[w]=l[v]+longitud
                π[w]=v
            end
        end
    end
    #paso 3
    conCircuito=0
    for c in 1:g.nArcos
        v=g.arcos[c].origen
        w=g.arcos[c].destino
        longitud=g.arcos[c].longitud
        if l[w]> (l[v]+longitud)
            conCircuito=w
            break
        end
    end
    return conCircuito,l,π
end

BellmanFordSparse (generic function with 1 method)

In [8]:
#conCircuito,l,π=BellmanFordDense(G,1)
#println("conCircuito: ",conCircuito," l: ",l,"\t π \t",π)
@time(BellmanFordDense(G,1));
@time(BellmanFordSparse(gS,1));

  6.049673 seconds (10.99 k allocations: 470.653 KB)
  0.133434 seconds (5.50 k allocations: 259.228 KB)


La reconstrucción del camino requiere reseguir desde el vértice conCircuito hasta que se repita un vértice en la traza

In [45]:
gS2=generarGrafoSparse(25,0.75,10);

conCircuito,l,π = BellmanFordSparse(gS2,1)
while conCircuito==0
    for c in 1:gS2.nArcos
        gS2.arcos[c].longitud -= 1
    end
    conCircuito,l,π=BellmanFordSparse(gS2,1)
end
#tenemos un circuito
inCircuit=zeros(gS2.nVertices)
current=conCircuito
while inCircuit[current]==0
    inCircuit[current]=1
    current=π[current]
end
#empieza
print("circuito en: ",current," ")
curr=π[current]
while curr!=current
    print(curr," ")
    curr=π[curr]
end
println(".")

circuito en: 18 17 16 12 24 23 25 .


# Floyd-Warshall

El procedimiento para caminos extremos entre toda pareja de vértices es un algoritmo de programación dinámica que recuerda la multiplicación de matrices.

En formato denso, el algoritmo resulta trivial, en formato disperso es preferible computar Djikstra repetidas veces (existe una manera de evitar circuitos que no requiere n ejecuciones de Bellman-Ford).

<b>P1. Inicialización:</b> $l_{ij}:=c_{ij}$ para todo $(i,j)\in A$, $l_{ij}:=\infty$ para todo $(i,j)\notin A$, $l_{ii}=0$ para todo $i\in V$, $\pi_{ij}=i$ para todo $(i,j)\in A$. 

<b>P2. Iteración:</b> 

Para j=1,...,|V|<br>
$~~~$Para i=1,...,|V|, si $j\neq i$<br>
$~~~~~~$Para todo k=1,...,|V|, si $k\neq j$<br>
$~~~~~~~~~$Si $l_{ik}>l_{ij}+l_{jk}$ entonces <br>
$~~~~~~~~~~~~$Sea $l_{ik}:=l_{ij}+l_{jk}$ y $p_{ik}:=p_{jk}$ <br>
$~~~~~~~~~$Fin Si <br>
$~~~~~~$Fin Para <br>
$~~~$Fin Para <br>
Fin Para

In [60]:
function FloydWarshallDense(g::gPaths)
    #paso 1
    ∞=1000000
    π=zeros(Int64,g.nVertices,g.nVertices)
    l=Array{Int64}(g.nVertices,g.nVertices)
    l=deepcopy(g.arcos)
    for i in 1:g.nVertices
        l[i,i]=0
    end
    for i = 1:g.nVertices
        for j = 1:g.nVertices
            π[i,j]=i
        end
    end

    #paso 2
    for j in 1:g.nVertices
        for i in 1:g.nVertices
            if i!=j
                for k in 1:g.nVertices
                    if k!=j
                        if l[i,k]>l[i,j]+l[j,k]
                            l[i,k]=l[i,j]+l[j,k]
                            π[i,k]=π[j,k]
                        end
                    end
                end
            end
        end
    end
    return l,π
end

function FloydWarshallSparse(g::gPathsSparse)
    #paso 1
    ∞=1000000
    π=zeros(Int64,g.nVertices,g.nVertices)
    l=Array{Int64}(g.nVertices,g.nVertices)
    fill!(l,∞)
    for i in 1:g.nArcos
        l[g.arcos[i].origen,g.arcos[i].destino]=g.arcos[i].longitud
    end
    for i in 1:g.nVertices
        l[i,i]=0
    end
    for i = 1:g.nVertices
        for j = 1:g.nVertices
            π[i,j]=i
        end
    end
    #paso 2
    for j in 1:g.nVertices
        for i in 1:g.nVertices
            if i!=j
                for k in 1:g.nVertices
                    if k!=j
                        if l[i,k]>l[i,j]+l[j,k]
                            l[i,k]=l[i,j]+l[j,k]
                            π[i,k]=π[j,k]
                        end
                    end
                end
            end
        end
    end
    return l,π
end

FloydWarshallSparse (generic function with 1 method)

In [61]:
@time(FloydWarshallDense(G));
@time(FloydWarshallSparse(gS));

 16.416649 seconds (12.39 k allocations: 23.373 MB, 0.01% gc time)
 15.717187 seconds (15.09 k allocations: 15.837 MB)
